# Import Lib

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, MultiHeadAttention, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
import os
import time

# Set the configuration for ViT structure.

In [ ]:
# กำหนดค่าคงที่พื้นฐานสำหรับโครงสร้าง ViT
IMAGE_SIZE = 224
PATCH_SIZE = 16
NUM_LAYERS = 8
EMBED_DIM = 384
MLP_DIM = 4 * EMBED_DIM
NUM_HEADS = 8
NUM_CLASSES = 1000 
DROPOUT_RATE = 0.1

# Vision Transformer from scracth

In [ ]:
class EncoderBlock(Model):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout_rate=0.1, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.mlp_dim = mlp_dim
        self.dropout_rate = dropout_rate

        self.norm1 = LayerNormalization(epsilon=1e-6, name="norm1")
        self.mha = MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim // num_heads,
            dropout=dropout_rate,
            name="multi_head_attention"
        )
        self.dropout_mha_output = Dropout(dropout_rate)
        self.norm2 = LayerNormalization(epsilon=1e-6, name="norm2")
        self.mlp = Sequential([
            Dense(mlp_dim, activation='gelu', name="mlp_dense_1"),
            Dropout(dropout_rate),
            Dense(embed_dim, name="mlp_dense_2"),
            Dropout(dropout_rate)
        ], name="mlp_block")

    def call(self, inputs, training=False):
        x_norm1 = self.norm1(inputs)
        attn_output = self.mha(query=x_norm1, value=x_norm1, key=x_norm1, training=training)
        attn_output_dropped = self.dropout_mha_output(attn_output, training=training)
        x_res1 = inputs + attn_output_dropped
        x_norm2 = self.norm2(x_res1)
        mlp_output = self.mlp(x_norm2, training=training)
        x_res2 = x_res1 + mlp_output
        return x_res2
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'mlp_dim': self.mlp_dim,
            'dropout_rate': self.dropout_rate,
        })
        return config

class VisionTransformer(Model):
    def __init__(self, image_size=IMAGE_SIZE, num_classes=NUM_CLASSES, patch_size=PATCH_SIZE, embed_dim=EMBED_DIM,
                 num_heads=NUM_HEADS, num_layers=NUM_LAYERS, mlp_dim=MLP_DIM, dropout_rate=0.1, **kwargs):
        super(VisionTransformer, self).__init__(**kwargs)
        self.num_classes = num_classes
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.mlp_dim = mlp_dim
        self.dropout_rate = dropout_rate
        self.num_patches = (image_size // patch_size) ** 2

        self.cls_token = self.add_weight(name="cls_token", shape=[1, 1, embed_dim], initializer=tf.keras.initializers.RandomNormal(stddev=0.02), trainable=True)
        self.pos_embed = self.add_weight(name="position_embedding", shape=[1, self.num_patches + 1, embed_dim], initializer=tf.keras.initializers.RandomNormal(stddev=0.02), trainable=True)
        self.pos_dropout = Dropout(dropout_rate)
        self.patch_embed = Conv2D(filters=embed_dim, kernel_size=patch_size, strides=patch_size, padding='valid', name="patch_embed")
        self.encoder_layers = [EncoderBlock(embed_dim, num_heads, mlp_dim, dropout_rate) for _ in range(num_layers)]
        self.norm_head = LayerNormalization(epsilon=1e-6, name="head_norm")
        self.head = Dense(num_classes, activation='softmax', name="classification_head")

    def call(self, inputs, training=False):
        batch_size = tf.shape(inputs)[0]
        x = self.patch_embed(inputs)
        x = tf.reshape(x, (batch_size, -1, self.embed_dim))
        cls_tokens = tf.tile(self.cls_token, [batch_size, 1, 1])
        x = tf.concat([cls_tokens, x], axis=1)
        x = x + self.pos_embed
        x = self.pos_dropout(x, training=training)
        for encoder in self.encoder_layers:
            x = encoder(x, training=training)
        cls_token_output = x[:, 0]
        cls_token_output = self.norm_head(cls_token_output, training=training)
        logits = self.head(cls_token_output, training=training)
        return logits
        
    def get_config(self):
        config = super().get_config()
        config.update({
            'image_size': self.num_patches**0.5 * self.patch_size, # Reconstruct from num_patches
            'num_classes': self.num_classes,
            'patch_size': self.patch_size,
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'num_layers': self.num_layers,
            'mlp_dim': self.mlp_dim,
            'dropout_rate': self.dropout_rate,
        })
        return config




# WarmupCosineDecay

In [ ]:
class WarmupCosineDecay(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, initial_learning_rate, decay_steps, warmup_steps, alpha=0.0, name=None):
        super().__init__()
        self.name = name
        self.initial_learning_rate = initial_learning_rate
        self.decay_steps = decay_steps
        self.warmup_steps = warmup_steps
        self.alpha = alpha
        self.cosine_decay_schedule = tf.keras.optimizers.schedules.CosineDecay(
            initial_learning_rate=self.initial_learning_rate,
            decay_steps=self.decay_steps - self.warmup_steps,
            alpha=self.alpha
        )

    def __call__(self, step):
        step_float = tf.cast(step, tf.float32)
        warmup_steps_float = tf.cast(self.warmup_steps, tf.float32)
        def warmup_fn():
            return (self.initial_learning_rate / warmup_steps_float) * step_float
        def cosine_decay_fn():
            return self.cosine_decay_schedule(step_float - warmup_steps_float)
        learning_rate = tf.cond(step_float < warmup_steps_float, warmup_fn, cosine_decay_fn)
        return learning_rate

    def get_config(self):
        return {
            "initial_learning_rate": self.initial_learning_rate,
            "decay_steps": self.decay_steps,
            "warmup_steps": self.warmup_steps,
            "alpha": self.alpha,
            "name": self.name
        }

# Predict a precision , recall and F1-scores

In [ ]:
# =======================================================================

# --- 1. กำหนดค่าและ Path ที่จำเป็น ---

datapath = '/media/capybara/Data/dataset_vit/archive'
best_model_path = '/media/capybara/Data/dataset_vit/codeVIT/best_vit_model_1000_classes.keras'
img_height = 224
img_width = 224
batch_size = 32

# --- 2. โหลดโมเดลและชุดข้อมูล Validation ---

print(f"กำลังโหลดโมเดล ViT จาก: {best_model_path}")
if not os.path.exists(best_model_path):
    raise FileNotFoundError(f"ไม่พบไฟล์โมเดลที่ '{best_model_path}'. กรุณาตรวจสอบ Path ให้ถูกต้อง")

# เพิ่ม custom_objects ตอนโหลดโมเดล
model = tf.keras.models.load_model(
    best_model_path,
    custom_objects={
        "VisionTransformer": VisionTransformer,
        "EncoderBlock": EncoderBlock,
        "WarmupCosineDecay": WarmupCosineDecay
    }
)
print("โหลดโมเดลสำเร็จ")

print(f"กำลังโหลดชุดข้อมูล Validation จาก: {datapath}")
val_dataset = tf.keras.utils.image_dataset_from_directory(
    datapath,
    validation_split=0.04,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)

class_names = val_dataset.class_names
print(f"พบ {len(class_names)} คลาส")

def preprocess_val_data(images, labels):
    images = images / 255.0  
    return images, labels

val_pipeline = val_dataset.map(preprocess_val_data, num_parallel_calls=tf.data.AUTOTUNE)
val_pipeline = val_pipeline.prefetch(buffer_size=tf.data.AUTOTUNE)

# --- 3. ทำนายผล (Predict) และรวบรวมผลลัพธ์ ---
y_pred = []
y_true = []
iteration = 0
total_batches = len(val_pipeline)

print(f"\nกำลังทำนายผลจากข้อมูล Validation ทั้งหมด {total_batches} batches...")
start_time = time.time()

for images, labels in val_pipeline:
    predictions = model.predict(images, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(labels.numpy(), axis=1)
    y_pred.extend(predicted_classes)
    y_true.extend(true_classes)
    
    iteration += 1
    if iteration % 50 == 0:
        print(f"  Processed {iteration}/{total_batches} batches...")

end_time = time.time()
print(f"ทำนายผลเสร็จสิ้นใน {end_time - start_time:.2f} วินาที")

# --- 4. คำนวณและแสดงผลลัพธ์ ---
y_pred = np.array(y_pred)
y_true = np.array(y_true)

print("\n" + "="*70)
print("            ผลการประเมินโมเดล (Classification Report)          ")
print("="*70 + "\n")

present_labels = np.unique(y_true)
print(f"พบข้อมูลจริง {len(present_labels)} คลาสในชุด Validation ที่ใช้ประเมินผล")

present_class_names = [class_names[i] for i in present_labels]

report = classification_report(
    y_true, 
    y_pred, 
    labels=present_labels, 
    target_names=present_class_names,
    zero_division=0,
    digits=4
)
print(report)

print("\n--- สรุปค่าเฉลี่ยประสิทธิภาพโมเดล ---")
report_dict = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

print("\n[ แนะนำ ] ค่าเฉลี่ยแบบถ่วงน้ำหนัก (Weighted Average)")
print(f"  - Precision: {report_dict['weighted avg']['precision']:.4f}")
print(f"  - Recall:    {report_dict['weighted avg']['recall']:.4f}")
print(f"  - F1-Score:  {report_dict['weighted avg']['f1-score']:.4f}")

print("\nค่าเฉลี่ยแบบไม่ถ่วงน้ำหนัก (Macro Average)")
print(f"  - Precision: {report_dict['macro avg']['precision']:.4f}")
print(f"  - Recall:    {report_dict['macro avg']['recall']:.4f}")
print(f"  - F1-Score:  {report_dict['macro avg']['f1-score']:.4f}")

print("\nความแม่นยำโดยรวม (Overall Accuracy)")
print(f"  - Accuracy: {report_dict['accuracy']:.4f}")
print("="*40)